### todo
- [ ] numpy, vectorisation (unless not possible)?
      ^ i guess it's not exactly possible

In [28]:
%pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Data example is from Morteza Rahmani https://www.researchgate.net/publication/335421455_Examining_the_impact_of_transfers_in_pickup_and_delivery_systems


**Data requirements**

*   Each vehicle has to start and end it's journey on a different node.




In [29]:
import numpy as np

def NO_TRANSFERS_1():
    data = {}
    # number of nodes in graph 
    # travel time between nodes
    # c[i][j] = -1 means that there is no connection
    data['c'] = np.array([[0  ,300,150,300,300 ,0  ,300],
                           [300,0  ,150,300,300,300,300],
                           [150,150,0  ,150,150,150,150],
                           [300,300,150,0  ,300,300,300],
                           [300,300,150,300,0  ,300,0  ],
                           [0  ,300,150,300,300 ,0 ,300],
                           [300,300 ,150,300,300,300,0]])
    data['N'] = 7
    data['avg_speed'] = 60


    # vehicles with start and end
    data['vehicles'] = [[0,5],[4,6]]     
    data['vehicles_capacity'] = [20,20]
    data['requests'] = [[0,1],[0,3],[4,1],[4,3]] 
    data['requests_size'] =[10,10,10,10]
    data['transfers'] = []
    data['time_window'] = [[0,100],[0,100],[0,100],[0,100],[0,100],[0,100],[0,100]]

    return data

def TRANSFERS_1():
    data = {}
    # number of nodes in graph 
    # travel time between nodes
    # c[i][j] = -1 means that there is no connection
    data['c'] = np.array([[0  ,300,150,300,300 ,0  ,300],
                           [300,0  ,150,300,300,300,300],
                           [150,150,0  ,150,150,150,150],
                           [300,300,150,0  ,300,300,300],
                           [300,300,150,300,0  ,300,0  ],
                           [0  ,300,150,300,300 ,0 ,300],
                           [300,300 ,150,300,300,300,0]])
    data['N'] = 7
    data['avg_speed'] = 60

    # vehicles with start and end
    data['vehicles'] = [[0,5],[4,6]]     
    data['vehicles_capacity'] = [20,20]
    data['requests'] = [[0,1],[0,3],[4,1],[4,3]] 
    data['requests_size'] =[10,10,10,10]
    data['transfers'] = [2]
    data['time_window'] = [[0,100],[0,100],[0,100],[0,100],[0,100],[0,100],[0,100]]

    return data

In [30]:
def non_transfer(transfers):
  return [i for i in range(0, data['N']) if i not in transfers]

In [31]:
def var_name(letter, indexies):
  return letter + "_" + "_".join(map(str, indexies))

$$\delta^+_k(S) = \{(u,v) \in A^k | u \in S, v\notin S\}$$

$$\delta^-_k(S) = \{(u,v) \in A^k | u \notin S, v\in S\}$$

In [32]:
def leaving_nodes(S):
    return [(u, v) for v in range(0, data["N"]) for u in S if u != v and data["c"][u][v] >= 0]

In [33]:
def entering_nodes(S):
    return [(u, v) for u in range(0, data["N"]) for v in S if u != v and data["c"][u][v]]

In [34]:
from ortools.linear_solver import pywraplp 

In [35]:
# solver = pywraplp.Solver.CreateSolver('SCIP')
solver = pywraplp.Solver('SCIP', pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)
if not solver:
    raise Exception("Failed to create the solver!")

**MIP VARIABLES**
$$s_v^k \text{- start time of service of vehicle $k \in K$ at node $v$}$$

$$x_{i,j}^k = \begin{cases}
    1, & \text{iff vehicle k $\in K$ traverses arc $(i,j) \in A$}.\\
    0, & \text{otherwise}.
  \end{cases}
  \\\\
y_v^k = \begin{cases}
    1, & \text{iff vehicle k $\in K$ visits vertex $v \in V$}.\\
    0, & \text{otherwise}.
\end{cases}
\\\\
f_{i,j}^{kr} = \begin{cases}
    1, & \text{iff vehicle k traverses arc $(i,j)$ with request $r$}.\\
    0, & \text{otherwise}.
  \end{cases}
\\\\
z^{kk'}_{vr} = \begin{cases}
    1, & \text{iff request r is transhipped from $k$ to $k'$ at node $v \in T$}.\\
    0, & \text{otherwise}.
  \end{cases}$$

In [36]:
data = TRANSFERS_1()

In [37]:
# creating linear variables

V = [v for v in range(data['N'])]
K = [k for k in range(len(data['vehicles']))]
R = [r for r in range(len(data['requests']))]
A = [(i,j) for i in range(data['N']) for j in range(data['N'])]
T = [t for t in range(len(data['transfers']))]

transfers = data['transfers']
time = data['c'] / data['avg_speed']
vehicles = data['vehicles']
requests = data['requests']
transfers = data['transfers']
vehicles_capacity = data['vehicles_capacity']
request_size = data['requests_size']
time_windows = data['time_window']
c = data['c']
requests_tw =  [[0,100],[0,100],[0,100],[0,100]]

# big M jak się okazuje tylko w teorii musi być duże, a powinno być ogolnie najmniejsze jak może być, 
# bo solver nie ogarnia później można to zmienić też na sume  
M = np.sum(time)


infinity = solver.infinity()

# s[k][v]
s = [[solver.NumVar(0, infinity, var_name("s", [k,v])) for v in V] for k in K]
# x[k][i][j]
x = [[[solver.IntVar(0, 1, var_name("x", [k,i,j])) for j in V ] for i in V] for k in K]
# y[k][v]
y = [[solver.IntVar(0, 1, var_name("y", [k,v])) for v in V] for k in K]
# f[k][r][i][j]
f = [[[[solver.IntVar(0, 1, var_name("f", [k,r,i,j])) for j in V] for i in V] for r in R] for k in K]
# z[k][k'][v][r]
z = [[[[solver.IntVar(0, 1, var_name("z", [k,k2,v,r])) for r in R] for v in V] for k2 in K] for k in K]

Objective function to minimize
$$min \sum_{k\in K}\sum_{(i,j)\in A}c_{i,j}x^k_{i,j} $$

In [38]:
objective = solver.Objective()
for k in K:
  for (i,j) in A:
    objective.SetCoefficient(x[k][i][j], int(c[i][j]))


1. Flow conservation
$$x^k(\delta^+_k(v)) = y_v^k ;\;\;\;\;\;\;\;\;\ \forall k \in K, \forall v \in V-\{k^-\}$$
$$\sum_{(i,j) \in \delta_k^+(v)}x_{i,j}^k = y_v^k$$

$$x^k(\delta^-_k(v)) = y_v^k 
\;\;\;\;\;\;\;\;\ \forall k \in K, \forall v \in V-\{k^+\}$$
$$\sum_{(i,j)\in \delta_k^-(v)}x_{i,j}^k = y_v^k$$

In [39]:
for k in K:
  for v in V:
    if v == vehicles[k][1]:
      solver.Add(sum(x[k][i][j] for (i,j) in leaving_nodes([v])) == 0)
    if v == vehicles[k][0]:
      solver.Add(sum(x[k][i][j] for (i,j) in entering_nodes([v])) == 0)
    if v != vehicles[k][1]:
      solver.Add(sum(x[k][i][j] for (i,j) in leaving_nodes([v])) == y[k][v])
    if v != vehicles[k][0]:
      solver.Add(sum(x[k][i][j] for (i,j) in entering_nodes([v])) == y[k][v])

for k in K:
  for i in V:
    solver.Add(x[k][i][i] == 0)

*2*. Guarantee that every vehicle starts and ends at it’s depot
$$y^k_v = 1 \;\;\;\;\;
 \forall
 k \in K, \forall v \in \{k^+,k^-\} $$

In [40]:
for k in K:
  # chosing first instance of copy for [k^+, k^-] pair
  k_origin = vehicles[k][0]
  k_destination = vehicles[k][1]
  solver.Add(y[k][k_origin] == 1)
  solver.Add(y[k][k_destination] == 1)




3. Guarantee that every request is served only once
$$\sum_{k \in K}f^k_r(\delta^+_k(V_{r^+})) =\sum_{k \in K}f^k_r(\delta^-_k(V_{r^-})) = 1 \;\;\;\; \forall r\in R$$
$$\sum_{k \in K}\sum_{(i,j) \in \delta_k^+(V_{r^+})} f^{kr}_{i,j} = 
\sum_{k \in K}\sum_{(i,j) \in \delta_k^-(V_{r^-})} f^{kr}_{i,j} = 1$$




In [41]:
for r in R:
  solver.Add(sum(sum(f[k][r][i][j] for k in K) for (i,j) in leaving_nodes([requests[r][0]])) == 1)
  solver.Add(sum(sum(f[k][r][i][j] for k in K) for (i,j) in entering_nodes([requests[r][1]])) == 1)


4. This forbids requests to enter (leave) it's origin (destination)

$$\sum_{k \in K}f^k_r(\delta^-_k(V_{r^+})) =\sum_{k \in K}f^k_r(\delta^+_k(V_{r^-})) = 0 \;\;\;\; \forall r\in R$$
$$\sum_{k \in K}\sum_{(i,j) \in \delta_k^-(V_{r^+})} f^{kr}_{i,j} = 
\sum_{k \in K}\sum_{a \in \delta_k^+(V_{r^-})} f^{kr}_{i,j}
 = 0$$

In [42]:
for r in R:
  solver.Add(sum(sum(f[k][r][i][j] for k in K) for (i,j) in entering_nodes([requests[r][0]])) == 0)
  solver.Add(sum(sum(f[k][r][i][j] for k in K) for (i,j) in leaving_nodes([requests[r][1]])) == 0)


5. This guarantee flow at transition points
$$f_r^k(\delta_k^-(v)) + \sum_{k' \in K}z^{k'k}_{vr} -z^{kk'}_{vr} = f_r^k(\delta^+_k(v))$$
\
$$\forall r \in R, t \in T, k \in K $$

In [43]:
for r in R:
  for t in transfers:
    for k in K:
      solver.Add(sum(f[k][r][i][j] for (i,j) in entering_nodes([t])) + sum(z[k2][k][t][r] - z[k][k2][t][r] for k2 in K) == 
                 sum(f[k][r][i][j] for (i,j) in leaving_nodes([t])))

7. This forbids transfers at locations other then transfer

$$\sum_{(i,j) \in \delta^+_k(v)}f^{kr}_{i,j} = \sum_{(i,j) \in \delta^-_k(v)}f^{kr}_{i,j}  $$
$$\forall r \in R, \forall k \in K, \forall v \in V - \{r^-,r^+\} - T $$

In [44]:
for r in R:
  for k in K:
    for v in V:
      if v not in transfers and v != requests[r][0] and v != requests[r][1]:
        solver.Add(sum(f[k][r][i][j] for (i,j) in leaving_nodes([v])) ==
                   sum(f[k][r][i][j] for (i,j) in entering_nodes([v])))

8. Time consistency

$$s_u^k + t_{u,v} - M(1-x^k_{u,v}) \leq s^k_v
 \;\;\; \forall k \in K,  \forall (u,v) \in A $$

In [45]:
for k in K:
  for (u,v) in A:
    solver.Add(s[k][u] + time[u][v] - M * (1 - x[k][u][v]) <= s[k][v]+1)

10. Time consistency at transport location
$$s_t^k  - M(1-z^{kk'}_{tr}) \leq s^{k'}_t
 \;\;\; \forall t \in T, \forall k,k' \in K $$

In [46]:
for t in T:
  for k in K:
    for k2 in K:
      for r in R:
        solver.Add(s[k][t] - M * (1 - z[k][k2][t][r]) <= s[k2][t])

11. This allows transhipment at $t \in T$ only when both vehicles arrive at $t$.

$$z_{tr}^{kk'} \leq y_t^k \;\;\;\; z_{tr}^{kk'} \leq y_t^{k'} $$
$$\forall t \in T,k,k' \in K,r \in R$$

In [47]:
for t in T:
  for k in K:
    for k2 in K:
      for r in R:
        solver.Add(z[k][k2][t][r] <= y[k][t])
        solver.Add(z[k][k2][t][r] <= y[k2][t])

12. Binds $f$ and $x$ variable
$$f^{kr}_{ij} \leq x^k_{ij}$$

In [48]:
for k in K:
  for (i,j) in A:
    for r in R:
      solver.Add(f[k][r][i][j] <= x[k][i][j])

13. Capacity constraint
$$\sum_{r \in R}f^{kr}_{i,j} r_k \leq k_r $$

In [49]:
for (i,j) in A:
  for k in K:
    solver.Add(sum(f[k][r][i][j] * request_size[r] for r in R) <= vehicles_capacity[k] )

14. Time Windows
$$ a_v \leq s^k_v  \leq b_v$$

In [50]:
for k in K:
  for v in non_transfer(transfers):    
    solver.Add(time_windows[v][1] <= s[k][v] <= time_windows[v][1])

In [51]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print("ok")
  print(solver.Objective().Value())
elif status == pywraplp.Solver.NOT_SOLVED:
  print("not solved")
elif status == pywraplp.Solver.INFEASIBLE:
  print("infeasible")

print("requests")
print(requests)
print("vehicles")
print(vehicles)
print("transfers")
print(transfers)

print("x variable")
for k in K:
  print("vehicle: "+ str(k) )
  for (i,j) in A:
    if x[k][i][j].solution_value() > 0:
      print(x[k][i][j], c[i][j])

print("y variable")
for k in K:
  for v in V:
    if y[k][v].solution_value() > 0:
      print(y[k][v])

print("f variable")
for k in K:
  for r in R:
    for (i,j) in A:
      if f[k][r][i][j].solution_value() > 0:
        print(f[k][r][i][j])

print("\n")
print("Time: ")
print(time)
for k in K:
  print("vehicle: "+ str(k))
  for v in V:
    print(s[k][v],end = " = ")
    print("{0:.90f}".format(s[k][v].solution_value()))

for k in K:
  for k2 in K:
    for v in V:
      for r in R:
        if z[k][k2][v][r].solution_value() > 0:
          print(z[k][k2][v][r])

ok
1200.0
requests
[[0, 1], [0, 3], [4, 1], [4, 3]]
vehicles
[[0, 5], [4, 6]]
transfers
[2]
x variable
vehicle: 0
x_0_0_2 150
x_0_2_3 150
x_0_3_5 300
vehicle: 1
x_1_1_6 300
x_1_2_1 150
x_1_4_2 150
y variable
y_0_0
y_0_2
y_0_3
y_0_5
y_1_1
y_1_2
y_1_4
y_1_6
f variable
f_0_0_0_2
f_0_1_0_2
f_0_1_2_3
f_0_3_2_3
f_1_0_2_1
f_1_2_2_1
f_1_2_4_2
f_1_3_4_2


Time: 
[[0.  5.  2.5 5.  5.  0.  5. ]
 [5.  0.  2.5 5.  5.  5.  5. ]
 [2.5 2.5 0.  2.5 2.5 2.5 2.5]
 [5.  5.  2.5 0.  5.  5.  5. ]
 [5.  5.  2.5 5.  0.  5.  0. ]
 [0.  5.  2.5 5.  5.  0.  5. ]
 [5.  5.  2.5 5.  5.  5.  0. ]]
vehicle: 0
s_0_0 = 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
s_0_1 = -0.000000000000000699906678184235441323079314651596209053614782748253819733008640469051897526
s_0_2 = 1.499999999999999777955395074968691915273666381835937500000000000000000000000000000000000000
s_0_3 = 3.000000000000000888178419700125232338905334472656250000000000000000000000000000000000000000
s_0_4 = 0.

In [52]:
def find_edge(edges, node):
  for edge in edges:
    if node == edge[0]:
      return edge

def get_vehicle_route(x, vehicles_num, start):
  route = {}
  edges = [(i, j) for (i, j) in A if x[vehicles_num][i][j].solution_value() > 0]

  route['edges'] = edges

  return route

In [53]:
def get_request_route(f, route_num, start):
  route = {}
  # is this good though?? wont the order get messed up?
  edges = [(i, j, k) for (i, j) in A for k in K if f[k][route_num][i][j].solution_value() > 0]

  route['edges'] = edges

  return route

Printing Solutions

In [54]:
vehicle_routes = [get_vehicle_route(x, k, vehicles[k][0]) for k in K]

for k, route in enumerate(vehicle_routes):
  print("vehicle: " + str(k),route['edges'], end = " ")
  # print(route['path'])

requests_routes = [get_request_route(f, r, requests[r][0]) for r in R]
print()

for r, route in enumerate(requests_routes):
  print("route: " + str(r)," starting at: ",requests[r][0], end = ": ")
  print(route['edges'])

vehicle: 0 [(0, 2), (2, 3), (3, 5)] vehicle: 1 [(1, 6), (2, 1), (4, 2)] 
route: 0  starting at:  0: [(0, 2, 0), (2, 1, 1)]
route: 1  starting at:  0: [(0, 2, 0), (2, 3, 0)]
route: 2  starting at:  4: [(2, 1, 1), (4, 2, 1)]
route: 3  starting at:  4: [(2, 3, 0), (4, 2, 1)]


Tutaj by się przydało wyświetlic rozwiazanie, np ,żeby na grafie wyświetlic w róznych kolorach ścieżki pojazdów poszczególnych a poźniej odzielnie kazdy request z scieżkami w kolorze odpowiadającym pojazdowi który go wiezie